# Data Cleaning



## Basic EDA
Observing columns and their datatypes:

In [1]:
import pandas as pd
raw_data = pd.read_csv('Life Expectancy Data.csv')
raw_data.sample(n=5).head()
raw_data.head()


,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [2]:
raw_data.columns

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')

In [3]:
#raw_data.info()

We immediately can see that there are some missing values and some zeros along with 'big' numeric values.
In addition, we have countries with 34 population, which is most definitely can't be.

In [4]:
# pd.options.display.float_format = '{:.2f}'.format
# raw_data.describe().T

## Data Cleaning

Renaming columns for easy access

In [5]:
raw_data.rename(columns = {'Country':'country',
                       'Year':'year',
                       'Status':'status',
                       'Life expectancy ':'life_expectancy',
                       'Adult Mortality':'adult_mortality',
                       'infant deaths':'infant_deaths',
                       'Alcohol':'alcohol',
                       'percentage expenditure':'percentage_expenditure',
                       'Hepatitis B':'hepatitis_b',
                       'Measles ':'measles',
                       ' BMI ':'bmi',
                       'under-five deaths ':'under-five deaths',
                       'Polio':'polio',
                       'Total expenditure':'total_expenditure',
                       'Diphtheria ':'diphtheria',
                       ' HIV/AIDS':'hiv/aids',
                       'GDP':'gdp',
                       'Population':'population',
                       ' thinness  1-19 years':'thinness_1_19_years',
                       ' thinness 5-9 years':'thinness_5_9_years',
                       'Income composition of resources':'income_composition_of_resources',
                       'Schooling':'schooling'
                       }, inplace = True, errors = 'raise')


## NA's & Outliers & Duplicates

First, checking for duplicates:

In [6]:
raw_data.duplicated().sum()

0

Nice, no duplicates!

Secondly, we will take a look of missing data in each row:

In [7]:
raw_data.isna().sum()

country                              0
year                                 0
status                               0
life_expectancy                     10
adult_mortality                     10
infant_deaths                        0
alcohol                            194
percentage_expenditure               0
hepatitis_b                        553
measles                              0
bmi                                 34
under-five deaths                    0
polio                               19
total_expenditure                  226
diphtheria                          19
hiv/aids                             0
gdp                                448
population                         652
thinness_1_19_years                 34
thinness_5_9_years                  34
income_composition_of_resources    167
schooling                          163
dtype: int64

Now, in alot of the cases - missing values comes as empty observations (a country without any valuable data). So firstly we want to observe the missing life expectancy:

In [8]:
#print(raw_data[raw_data['life_expectancy'].isnull()])

That's good - all of those countries does not have most of their data and relate to same year.So we can drop them without a doubt:

In [9]:
raw_data.drop(raw_data[raw_data['life_expectancy'].isnull()].index, axis=0, inplace=True)

### Exploring each variable
After droping some rows, we need to recheck which columns still have null values:


In [10]:
raw_data.isna().sum()

country                              0
year                                 0
status                               0
life_expectancy                      0
adult_mortality                      0
infant_deaths                        0
alcohol                            193
percentage_expenditure               0
hepatitis_b                        553
measles                              0
bmi                                 32
under-five deaths                    0
polio                               19
total_expenditure                  226
diphtheria                          19
hiv/aids                             0
gdp                                443
population                         644
thinness_1_19_years                 32
thinness_5_9_years                  32
income_composition_of_resources    160
schooling                          160
dtype: int64

#### Alcohol


In [11]:
#print(np.unique(raw_data[raw_data['alcohol'].isnull()]['year'],return_counts=True))
column = ['alcohol']
raw_data.loc[:,column] = raw_data.loc[:,column].bfill()

All missing values in alcohol were of year 2015, so we could just back-fill them.

#### Hepatitis B

In [12]:
# raw_data[raw_data['hepatitis_b'].isnull()].describe().T
# raw_data[raw_data['hepatitis_b'].isnull()]

All of those observations relate to **developing** countries. Hence, we can back-fill them *in respect to developing countries*:

In [13]:
developing_mean = raw_data[raw_data['status']=='Developing'].mean(skipna=True, numeric_only=True).hepatitis_b
#back-filling:
hep_na_ind = raw_data[(raw_data['status']=='Developing') & (raw_data['hepatitis_b'].isnull())].index
for row in hep_na_ind:
    raw_data.at[row,'hepatitis_b'] = developing_mean

# the rest will be front filled:
column = ['hepatitis_b']
raw_data.loc[:,column] = raw_data.loc[:,column].ffill()

#### BMI

In [14]:
# import matplotlib.pyplot as plt
# raw_data[raw_data['bmi'].isnull()].describe().T
# raw_data[raw_data['bmi'].isnull()]
# plt.boxplot(raw_data['bmi'])
# plt.show()

#Dropping all column due too many outliers
raw_data.drop('bmi',axis=1, inplace=True)


#### Polio

In [15]:
# raw_data[raw_data['polio'].isnull()].describe().T
# raw_data[raw_data['polio'].isnull()]
# raw_data.boxplot(column='polio', return_type='axes')

column = ['polio']
raw_data.loc[:,column] = raw_data.loc[:,column].ffill()

#### Total Expenditure

In [16]:
#raw_data[raw_data['total_expenditure'].isnull()]

Note that beside "Democratic People's Republic of Korea" which miss almost all of it values, most of the countries miss only at 2015. So we can back-fill them.

In [17]:
raw_data.drop(raw_data[raw_data['country']== "Democratic People's Republic of Korea"].index)
column = ['total_expenditure']
raw_data.loc[:,column] = raw_data.loc[:,column].bfill()

#### Diphtheria

In [18]:
# raw_data[raw_data['diphtheria'].isnull()]
column = ['diphtheria']
raw_data.loc[:,column] = raw_data.loc[:,column].ffill()

#### GDP

In [19]:
# raw_data[raw_data['gdp'].isnull()]

There are some critical countries missing this important variable (USA for example). One option is just remove it. But notice that all countries missing gdp are missing at all 15 observations. That hint us to back-fill them deviding to status:

In [20]:
#calculating mean to fill
developing_mean = raw_data[raw_data['status'] == 'Developing'].mean(skipna=True, numeric_only=True).gdp
developed_mean = raw_data[raw_data['status']== 'Developed'].mean(skipna=True, numeric_only=True).gdp
# catching rows with null
rows_gdp_null_developing = raw_data[(raw_data['gdp'].isnull()) & (raw_data['status'] == 'Developing')].index
rows_gdp_null_developed = raw_data[(raw_data['gdp'].isnull()) & (raw_data['status'] == 'Developed')].index
#back-filling:
for row in rows_gdp_null_developing:
    raw_data.at[row,'gdp'] = developing_mean
for row in rows_gdp_null_developed:
    raw_data.at[row,'gdp'] = developed_mean

#### Population

In [21]:
#raw_data[raw_data['population'].isnull()]

Population is data type which is indepedent of status, and hard to impute

Therefore I decided to drop the column itself


In [22]:
raw_data.drop('population', axis=1, inplace=True)

#### Thinness

In [23]:
# raw_data[raw_data['thinness_1_19_years'].isnull()]
# raw_data[raw_data['thinness_5_9_years'].isnull()]

Same way, filling with mean calculated respect to developing countries:

In [24]:
deve_mean_1_19 = raw_data[raw_data['status'] == 'Developing'].mean(skipna=True, numeric_only=True).thinness_1_19_years
raw_data['thinness_1_19_years'] = raw_data['thinness_1_19_years'].fillna(deve_mean_1_19)

deve_mean_5_9 = raw_data[raw_data['status']== 'Developed'].mean(skipna=True, numeric_only=True).thinness_5_9_years
raw_data['thinness_5_9_years'] = raw_data['thinness_5_9_years'].fillna(deve_mean_5_9)

##### Income composition of resources & Schooling

In [25]:
#raw_data[raw_data['income_composition_of_resources'].isnull()]

Note that if one miss it's income values, it is also has NA at schooling. So we can fill them together similarly to GDP column:

In [26]:
country_index_d = raw_data[(raw_data['income_composition_of_resources'].isnull())].index

developing_income_mean = raw_data[raw_data['status']=='Developing'].mean(skipna=True, numeric_only=True).income_composition_of_resources
developing_school_mean = raw_data[raw_data['status']=='Developing'].mean(skipna=True, numeric_only=True).schooling

developed_income__mean = raw_data[raw_data['status']=='Developed'].mean(skipna=True, numeric_only=True).income_composition_of_resources
developed_school__mean = raw_data[raw_data['status']=='Developed'].mean(skipna=True, numeric_only=True).schooling

for i in country_index_d:
    if raw_data['status'][i] == 'Developed':
        raw_data.at[i, 'income_composition_of_resources'] = developed_income__mean
        raw_data.at[i, 'schooling'] = developed_school__mean
    if raw_data['status'][i] == 'Developing':
        raw_data.at[i, 'income_composition_of_resources'] = developing_income_mean
        raw_data.at[i, 'schooling'] = developing_school_mean

After all of this cleaning, SANITY CHECK:

In [27]:
raw_data.isnull().sum()

country                            0
year                               0
status                             0
life_expectancy                    0
adult_mortality                    0
infant_deaths                      0
alcohol                            0
percentage_expenditure             0
hepatitis_b                        0
measles                            0
under-five deaths                  0
polio                              0
total_expenditure                  0
diphtheria                         0
hiv/aids                           0
gdp                                0
thinness_1_19_years                0
thinness_5_9_years                 0
income_composition_of_resources    0
schooling                          0
dtype: int64

In [28]:
raw_data.to_csv('Life_expec_cleaned.csv',index = False)